# **Compliance GPT with LangChain**

## **Setup**

In [1]:
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()

True

## **Config**

In [2]:
from utils.config import get_config

config = get_config()

In [3]:
USER_ID = 'xmriz'
CONVERSATION_ID = 'xmriz-2021-07-01-01'

## **Define Model**

In [4]:
from utils.models import ModelName, LLMModelName, EmbeddingModelName, get_model

model_name = ModelName.AZURE_OPENAI
llm_model, embed_model = get_model(model_name=model_name, config=config, llm_model_name=LLMModelName.GPT_35_TURBO, embedding_model_name=EmbeddingModelName.EMBEDDING_ADA)

## **Load Vector Store Index**

In [5]:
from databases.vector_store import RedisIndexManager

redis = RedisIndexManager(index_name='ojk', embed_model=embed_model, config=config, db_id=0)
vector_store = redis.load_vector_index()

c:\Users\acer\miniconda3\envs\chatbot-ocbc\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## **Get Retriever**

In [6]:
from retriever.retriever_ojk.retriever_ojk import get_retriever_ojk

retriever = get_retriever_ojk(vector_store=vector_store, top_n=6, top_k=16, llm_model=llm_model, embed_model=embed_model, config=config)

## **Create Chain**

In [7]:
from constant.ojk.prompt import CONTEXTUALIZE_Q_PROMPT_STR, QA_SYSTEM_PROMPT_STR
from databases.chat_store import MongoDBChatStore, RedisChatStore
from chain.chain_ojk.chain_ojk import create_ojk_chain
from chain.rag_chain import create_chain_with_chat_history

chat_store = RedisChatStore(k=4, config=config, db_id=1)

chain = create_ojk_chain(
    contextualize_q_prompt_str=CONTEXTUALIZE_Q_PROMPT_STR,
    qa_system_prompt_str=QA_SYSTEM_PROMPT_STR,
    retriever=retriever,
    llm_model=llm_model,
    # chat_store=chat_store
)

chain_history = create_chain_with_chat_history(
    final_chain=chain,
    chat_store=chat_store
)

## **Evaluation**

#### **Get Response**

In [9]:
from chain.rag_chain import get_response

response = get_response(
    chain=chain_history,
    question="Berapa SWDKLLJ dari buldozer?",
    user_id=USER_ID,
    conversation_id=CONVERSATION_ID
)

response

{'rewrited question': 'Berapa besar premi asuransi SWDKLLJ yang harus dibayarkan oleh pemilik buldozer?',
 'answer': 'Besar premi asuransi SWDKLLJ yang harus dibayarkan oleh pemilik buldozer sebesar Rp 20.000,00 (dua puluh ribu rupiah).\n\nSource: Peraturan Menteri Keuangan Nomor 36/PMK.010/2008 tentang Besar Santunan dan Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan (https://www.ojk.go.id/id/regulasi/Documents/Pages/PMK-Nomor-36PMK.010-Tahun-2008-tentang-Besar-Santunan-dan-Sumbangan-Wajib-Dana-Kecelakaan-Lalu-Lintas-Jalan/menas13_1389258036.pdf#page=3)',
 'context': '[\n  {\n    "metadata": {\n      "id": "doc:ojk:0d063bb180c64673a070b97e6d024c25",\n      "title": "Peraturan Menteri Keuangan Nomor 36/PMK.010/2008 tentang Besar Santunan dan Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan",\n      "sector": "IKNB",\n      "subsector": "Asuransi",\n      "regulation_type": "Klasifikasi Bapepam",\n      "regulation_number": "36/PMK.010/2008",\n      "effective_date": "26 Februari 20

#### **Chat Store Monitor**

In [10]:
# to print chat history
print(chat_store.get_session_history(user_id=USER_ID, conversation_id=CONVERSATION_ID))

Human: Apa itu SWDKLLJ?
AI: SWDKLLJ merupakan premi asuransi yang dibayarkan oleh para pengusaha/pemilik alat angkutan lalu lintas jalan kepada perusahaan yang menyelenggarakan Dana Kecelakaan Lalu Lintas Jalan.

Source: [36/PMK.010/2008](https://www.ojk.go.id/id/regulasi/Documents/Pages/PMK-Nomor-36PMK.010-Tahun-2008-tentang-Besar-Santunan-dan-Sumbangan-Wajib-Dana-Kecelakaan-Lalu-Lintas-Jalan/menas13_1389258036.pdf#page=2)
Human: Berapa SWDKLLJ dari buldozer?
AI: Besar premi asuransi SWDKLLJ yang harus dibayarkan oleh pemilik buldozer sebesar Rp 20.000,00 (dua puluh ribu rupiah).

Source: Peraturan Menteri Keuangan Nomor 36/PMK.010/2008 tentang Besar Santunan dan Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan
https://www.ojk.go.id/id/regulasi/Documents/Pages/PMK-Nomor-36PMK.010-Tahun-2008-tentang-Besar-Santunan-dan-Sumbangan-Wajib-Dana-Kecelakaan-Lalu-Lintas-Jalan/menas13_1389258036.pdf#page=4
Human: Bursa Efek wajib menyampaikan laporan realisasi anggaran kepada Bapepam melalui de

In [11]:
# to make the chat history empty
chat_store.clear_all()

#### **Evaluation**

In [12]:
eval_questions = [
    "Apa itu SWDKLLJ?",
    "Bursa Efek wajib menyampaikan laporan realisasi anggaran kepada Bapepam melalui dewan komisaris dengan ketentuan apa?",
    "KEPUTUSAN KETUA BADAN PENGAWAS PASAR MODAL NOMOR KEP-06/PM/1996 tentang apa?",
    "Siapa yang menandatangani keputusan ketua bapepam nomor kep-07/PM/1996?",
    "Penolakan atas permohonan persetujuan mengenai pengajuan atau perubahan peraturan Lembaga Kliring dan Penjaminan dilakukan dengan menggunakan formulir nomor berapa?",
    "Anggota Dewan Pengawas Syariah Perusahaan Pembiayaan dilarang untuk apa?",
    "Berapa uang pertanggungan dan pertanggungan total loss only (tlo) untuk kategori 5?",
    "Perusahaan wajib menyampaikan laporan hasil pengawasan Dewan Pengawas Syariah sebagaimana dimaksud dalam Pasal 2 dalam bentuk apa, kepada siapa, dan paling lambat kapan?",
    "Batas tingkat solvabilitas minimum bagi perusahaan asuransi dan perusahaan reasuransi ditingkatkan berdasarkan apa?",
    "Kapan PERATURAN KETUA BADAN PENGAWAS PASAR MODAL DAN LEMBAGA KEUANGAN NOMOR: PER- 10/BL/2012 mulai berlaku?",
    "Apa itu Dana Tabarru’",
    "Dalam rangka mendorong pertumbuhan industri Dana Pensiun, dipandang perlu mengubah ketentuan mengenai sanksi administratif berupa apa?",
    "Laporan Tahunan dan Laporan Semester sekurang-kurangnya mencakup aspek apa saja?",
    "Pendiri harus mengajukan permohonan tertulis kepada siapa untuk mendapatkan pengesahan pendirian Dana Pensiun Lembaga Keuangan?",
    "Berapa tarif pemotongan pajak untuk penghasilan bruto di atas seratus juta sampai dengan dua ratus juta rupiah?",
    "Apa yang dimaksud dengan 'telah beroperasi'?",
    "Lembaga Jasa Keuangan Non-Bank yang tidak memenuhi ketentuan Pasal 2 dan Pasal 3 dalam Peraturan OJK ini, dikenakan sanksi administratif berupa apa saja?",
    "Apa yang dimaksud dengan Lembaga Sertifikasi Profesi?",
    "Apa pertimbangan profil risiko perusahaan?",
    "LJKNB yang memiliki total aset lebih dari Rp500.000.000.000,00 (lima ratus miliar rupiah) sampai dengan Rp1.000.000.000.000,00 (satu triliun rupiah) wajib untuk apa?",
    "Siapa presiden republik Indonesia yang menandatangani peraturan pemerintah republik Indonesia nomor 12 Tahun 2004?",
    "Berapa jumlah modal yang disetor bursa efek?",
    "Bagaimana norma pemeriksaan yang menyangkut pihak yang diperiksa?",
    "Apa isi Pasal II PERATURAN PEMERINTAH REPUBLIK INDONESIA NOMOR 63 TAHUN 1999?",
    "Berapa modal yang disetor bagi perusahaan Asuransi Jiwa yang seluruh pemiliknya warga negara Indonesia?",
    "Apa itu Bank Perkreditan Rakyat?",
    "Bagaimana pelaksanaan wawancara jika calon pemegang saham pengendali BPR adalah pemerintah?",
    "Berapa modal yang disetor untuk mendirikan Bank Umum?",
    "Bagaimana cara bank untuk meningkatkan good corporate governance?",
    "Peraturan Bank Indonesia Nomor 7/33/PBI/2005 berisi pencabutan atas peraturan bank indonesia nomor berapa dan tentang apa?",
    "Surat edaran bank indonesia no 15/14/DPNP ditujukan kepada siapa?",
    "Apa isi ketentuan peralihan dari surat edaran No. 15/20/DKBU?",
    "Surat edaran bank indonesia No. 15/22/DPbS mencabut surat edaran bank indonesia nomor berapa?",
    "Kapan dan di mana Surat Edaran No. 15/25/DPNP mulai berlaku?",
    "Bank Syariah wajib membentuk cadangan kerugian penurunan nilai (CKPN) atas apa?",
    "Berapa penurunan IHSG BEI sejak tanggal 20 Mei 2013-27 Agustus 2013?",
    "Berapa lama Pengguna S-INVEST yang bertindak sebagai Bank Kustodian melakukan uji coba penyampaian surat?",
    "Apa judul regulasi nomor 1/SEOJK.04/2002?",
    "Penilaian Faktor Profil Risiko, menjelaskan mengenai mekanisme penilaian faktor profil risiko, yang terdiri dari 9 jenis risiko, apa saja jenis risiko tersebut?",
    "Sumber daya merupakan kriteria dalam melakukan penilaian terhadap lembaga pemeringkat yang digunakan untuk apa?",
    "Asuransi dapat diberikan dalam bentuk apa saja?",
    "Badan Supervisi terdiri dari berapa anggota?",
    "Apa yang dimaksud dengan Dana Pensiun berdasarkan pasal 134 UU PPSK?",
    "Tahun berapa krisis keuangan terbesar di Amerika Serikat pernah terjadi?",
    "Siapa menteri keuangan yang mensahkan UU RI Nomor 84 Tahun 1958?"
]

eval_answers = [
    "Sumbangan Wajib Dana Kecelakaan Lalu Lintas adalah sumbangan wajib sebagaimana dimaksud dalam Undang-Undang Nomor 34 Tahun 1964 tentang Dana Kecelakaan Lalu Lintas Jalan juncto Peraturan Pemerintah Nomor 18 Tahun 1965 tentang Ketentuan-Ketentuan Pelaksanaan Dana Kecelakaan Lalu Lintas Jalan",
    "Bursa Efek wajib menyampaikan laporan realisasi anggaran kepada Bapepam melalui dewan komisaris, dengan ketentuan bahwa laporan tersebut disampaikan secara kumulatif triwulanan dan diterima oleh Bapepam selambat-lambatnya pada hari ke dua belas setelah berakhirnya triwulan yang bersangkutan",
    "TATA CARA PEMBERIAN PERSETUJUAN ANGGARAN DASAR BURSA EFEK",
    "I Putu Gede Ary Suta, Badan Pengawas Pasar Modal",
    "Penolakan atas permohonan persetujuan mengenai pengajuan atau perubahan peraturan Lembaga Kliring dan Penjaminan dilakukan dengan menggunakan Formulir Nomor III.B.2-3 lampiran 3 peraturan ini.",
    "Dilarang melakukan rangkap jabatan sebagai anggota direksi atau dewan komisaris pada Perusahaan Pembiayaan, Dilarang merangkap jabatan sebagai anggota Dewan Pengawas Syariah lebih dari 2 (dua) Perusahaan Pembiayaan lain.",
    "Lebih dari Rp800.0000.000,00 dan 0,28%",
    "Perusahaan wajib menyampaikan laporan hasil pengawasan Dewan Pengawas Syariah sebagaimana dimaksud dalam Pasal 2 dalam bentuk dokumen fisik (hardcopy) dan format digital (softcopy) kepada Biro Perasuransian Badan Pengawas Pasar Modal dan Lembaga Keuangan, paling lambat tanggal 31 Maret tahun berikutnya.",
    "Batas tingkat solvabilitas minimum bagi perusahaan asuransi dan perusahaan reasuransi sebagaimana dimaksud dalam Pasal 3 Keputusan Menteri Keuangan Nomor 424/KMK.06/2003 tentang Kesehatan Keuangan Perusahaan Asuransi dan Perusahaan Reasuransi sebagaimana telah beberapa kali diubah terakhir dengan Peraturan Menteri Keuangan Nomor 158/PMK.010/2008, ditetapkan berdasarkan besarnya risiko kerugian yang mungkin timbul sebagai akibat dari deviasi dalam pengelolaan kekayaan dan kewajiban.",
    "Peraturan Ketua ini mulai berlaku pada tanggal 1 Juni 2013.",
    "Dana Tabarru’ adalah kumpulan dana yang berasal dari kontribusi para peserta, yang mekanisme penggunaannya sesuai dengan Akad Tabarru’ yang disepakati",
    "Dalam rangka mendorong pertumbuhan industri Dana Pensiun, dipandang perlu mengubah ketentuan mengenai sanksi administratif berupa denda atas keterlambatan penyampaian laporan investasi sebagaimana telah ditetapkan dalam Peraturan Menteri Keuangan Nomor 199/PMK.010/2008 tentang Investasi Dana Pensiun",
    "Operasional, keuangan, investasi, aktuaria, dan statistik",
    "Pendiri harus mengajukan permohonan tertulis kepada Menteri c.q. Ketua Badan Pengawas Pasar Modal dan Lembaga Keuangan u.p. Kepala Biro Dana Pensiun, Badan Pengawas Pasar Modal dan Lembaga Keuangan untuk mendapatkan pengesahan pendirian Dana Pensiun Lembaga Keuangan",
    "15%",
    "Yang dimaksud dengan “telah beroperasi” adalah telah mendapatkan izin usaha bagi perusahaan asuransi jiwa, perusahaan asuransi umum, perusahaan reasuransi, dan lembaga penjaminan termasuk yang menyelenggarakan seluruh atau sebagian usahanya dengan prinsip syariah, atau telah memperoleh pengesahan bagi dana pensiun pemberi kerja.",
    "Peringatan tertulis, penilaian kembali kemampuan dan kepatutan, larangan menjadi pemegang saham",
    "Lembaga Sertifikasi Profesi adalah lembaga pelaksana kegiatan sertifikasi profesi yang memperoleh lisensi dari lembaga negara yang berwenang memberikan lisensi terhadap lembaga sertifikasi profesi di Indonesia.",
    "Profil risiko Perusahaan mempertimbangkan antara lain letak geografis, produk Perusahaan, rencana bisnis, dan klaim experience dalam mengukur profil risiko serta mempertimbangkan hasil simulasi skenario perubahan (stress test). Selain itu Perusahaan dapat mempertimbangkan risiko lain yang mungkin timbul antara lain berasal dari rencana perubahan strategi dan/atau pengembangan bisnis Perusahaan.",
    "Memiliki pusat data dan melakukan rekam cadang data aktivitas yang diproses menggunakan Teknologi Informasi yang dilakukan secara berkala",
    "Megawati Soekarnoputri",
    "Modal disetor Bursa Efek sekurang-kurangnya berjumlah Rp7.500.000.000,00 (tujuh miliar lima ratus juta rupiah).",
    "Pihak yang diperiksa berhak meminta kepada Pemeriksa untuk memperlihatkan Surat Perintah Pemeriksaan dan Tanda Pengenal Pemeriksa, Pihak yang diperiksa berhak meminta kepada Pemeriksa untuk memberikan penjelasan tentang maksud dan tujuan pemeriksaan, dan Pihak yang diperiksa menandatangani surat pernyataan persetujuan tentang hasil pemeriksaan.",
    "Bagi permohonan izin usaha Perusahaan Asuransi dan Perusahaan Reasuransi yang telah diajukan dan yang telah memperoleh izin prinsip sebelum Peraturan Pemerintah ini ditetapkan, persyaratan permodalan tetap diberlakukan berdasarkan persyaratan yang berlaku pada saat izin prinsip ditetapkan.",
    "Rp. 2.000.000.000,-",
    "Bank Perkreditan Rakyat, yang selanjutnya disebut BPR, adalah Bank Perkreditan Rakyat sebagaimana dimaksud dalam Pasal 1 angka 4 Undang Undang Nomor 7 Tahun 1992 tentang Perbankan sebagaimana telah diubah dengan Undang-undang Nomor 10 Tahun 1998, yang melaksanakan kegiatan usaha secara konvensional.",
    "Dalam hal calon Pemegang Saham Pengendali BPR adalah pemerintah, maka pelaksanaan wawancara sebagaimana dimaksud dalam Pasal 8 ayat (1) huruf b hanya dilakukan apabila dianggap perlu.",
    "Modal disetor untuk mendirikan Bank ditetapkan sekurang-kurangnya sebesar Rp3.000.000.000.000,00 (tiga triliun rupiah).",
    "Bahwa dalam rangka meningkatkan good corporate governance, bank perlu menetapkan sasaran strategis dan seperangkat nilai perusahaan (corporate values) yang mengarahkan kegiatan operasional bank;",
    "PERATURAN BANK INDONESIA NOMOR 5/17/PBI/2003 TENTANG PERSYARATAN DAN TATA CARA PELAKSANAAN JAMINAN PEMERINTAH TERHADAP KEWAJIBAN PEMBAYARAN BANK PERKREDITAN RAKYAT",
    "Kepada SEMUA BANK UMUM YANG MELAKUKAN KEGIATAN USAHA SECARA KONVENSIONAL DI INDONESIA",
    "A. Penyusunan dan penyampaian Laporan Bulanan dan/atau Koreksi Laporan Bulanan untuk posisi bulan sebelum bulan Agustus 2013 tetap berpedoman pada Surat Edaran Bank Indonesia Nomor 8/7/DPBPR tanggal 23 Februari 2006 perihal Laporan Bulanan Bank Perkreditan Rakyat sebagaimana telah diubah terakhir dengan Surat Edaran Bank Indonesia Nomor 12/15/DKBU tanggal 11 Juni 2010. B. BPR Pelapor melakukan uji coba penyampaian Laporan Bulanan untuk posisi bulan Juni dan Juli 2013 yang masing-masing disampaikan paling lambat pada akhir bulan berikutnya, dengan berpedoman pada Surat Edaran Bank Indonesia ini.",
    "Dengan berlakunya Surat Edaran Bank Indonesia ini maka Surat Edaran Bank Indonesia Nomor 8/19/DPbS tanggal 24 Agustus 2006 perihal Pedoman Pengawasan Syariah dan Tata Cara Pelaporan Hasil Pengawasan bagi Dewan Pengawas Syariah dicabut dan dinyatakan tidak berlaku.",
    "Jakarta, 9 Juli 2013",
    "Bank Syariah wajib membentuk cadangan kerugian penurunan nilai (CKPN) atas aset keuangan dan aset non keuangan sesuai dengan standar akuntansi keuangan yang berlaku.",
    "Berdasarkan pertimbangan sebagaimana dimaksud angka I, maka penurunan Indeks Harga Saham Gabungan Bursa Efek Indonesia sejak tanggal 20 Mei 2013 sampai dengan tanggal 27 Agustus 2013 ini sebesar 1.247,134 poin atau 23,91% (dua puluh tiga koma sembilan satu perseratus) ditetapkan sebagai Kondisi Lain sebagaimana dimaksud Pasal 1 angka 1 huruf b POJK Nomor 2/POJK.04/2013.",
    "Pengguna S-INVEST yang bertindak sebagai Bank Kustodian melakukan uji coba penyampaian surat atau bukti konfirmasi dan laporan berkala Reksa Dana melalui S-INVEST dalam jangka waktu 12 (dua belas) bulan sejak ditetapkannya Surat Edaran Otoritas Jasa Keuangan ini",
    "SURAT EDARAN OTORITAS JASA KEUANGAN REPUBLIK INDONESIA NOMOR 1/SEOJK.04/2022 TENTANG TATA CARA PERMOHONAN PERSETUJUAN SEBAGAI PENYEDIA SISTEM PENYELENGGARAAN RAPAT UMUM PEMEGANG SAHAM SECARA ELEKTRONIK (E-RUPS)",
    "Risiko inheren atas risiko strategis, operasional, asuransi, kredit, pasar, likuiditas, hukum, kepatuhan, dan reputasi.",
    "Kriteria ini digunakan untuk menilai kemampuan lembaga pemeringkat dalam memberikan jasa pemeringkatan, baik dari aspek sumber daya manusia (human resources), aspek sumber daya keuangan (financial resources), maupun dukungan pemegang saham, yang memungkinkan lembaga pemeringkat beroperasi secara independen dan profesional.",
    "Asuransi atas risiko kegagalan Ekspor, atas risiko kegagalan bayar, atas investasi yang dilakukan oleh perusahaan Indonesia di luar negeri, dan/atau atas risiko politik di suatu negara yang menjadi tujuan ekspor",
    "Badan Supervisi terdiri 5 (lima) orang anggota terdiri dari seorang Ketua merangkap anggota, dan 4 (empat) orang anggota yang dipilih oleh Dewan Perwakilan Rakyat dan diangkat oleh Presiden untuk masa jabatan 3 (tiga) tahun dan dapat dipilih kembali untuk satu kali masa jabatan berikutnya.",
    "Berdasarkan Pasal 134 UU PPSK, Dana Pensiun adalah badan hukum yang mengelola dan menjalankan program yang menjanjikan manfaat pensiun. Selain manfaat pensiun, Dana Pensiun dapat menyelanggarakan manfaat lain sebagai manfaat tambahan sebagaimana diatur dalam peraturan perundang-undangan di bidang Dana Pensiun.",
    "Krisis keuangan akhir-akhir ini di Amerika Serikat yang merupakan terbesar sejak krisis 1929",
    "Soetikno Slamet"
]

In [13]:
import time
from datasets import Dataset
from chain.rag_chain import get_response

results = []
contexts = []
queries = eval_questions
ground_truths = eval_answers

for query in queries:
    result = get_response(
        chain=chain_history,
        question=query,
        user_id=USER_ID,
        conversation_id=CONVERSATION_ID
    )

    results.append(result['answer'])
    sources = result["context"]
    final_sources = list(eval(sources))
    contents = []
    for source in final_sources:
        contents.append(source.get('page_content'))
    contexts.append(contents)

    time.sleep(3)

d = {
    "question": queries,
    "answer": results,
    "contexts": contexts,
    "ground_truth": ground_truths
}

dataset = Dataset.from_dict(d)

Metadata key page_number not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['title', 'sector', 'subsector', 'regulation_type', 'regulation_number', 'effective_date', 'file_url', 'doc_id', 'page_number']
Metadata key page_number not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['title', 'sector', 'subsector', 'regulation_type', 'regulation_number', 'effective_date', 'file_url', 'doc_id', 'page_number']


OutputParserException: Parsing text
```json
{
    "query": "tarif pemotongan pajak",
    "filter": "and(gte(\"penghasilan_bruto\", \"100000000\"), lte(\"penghasilan_bruto\", \"200000000\"))"
}
```
 raised following error:
Received invalid attributes penghasilan_bruto. Allowed attributes are ['title', 'sector', 'subsector', 'regulation_type', 'regulation_number', 'effective_date']

In [13]:
import json

with open("./constant/ojk/dict_eval.json", "w") as outfile: 
    json.dump(d, outfile)

NameError: name 'd' is not defined

In [ ]:
with open('./constant/ojk/dict_eval.json') as json_file:
    d = json.load(json_file)

In [ ]:
dataset = Dataset.from_dict(d)

df = dataset.to_pandas()
df.head()

In [ ]:
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall, answer_similarity, answer_correctness
from ragas.metrics.critique import harmfulness
from ragas import evaluate

score = evaluate(dataset, metrics=[faithfulness, answer_relevancy, context_precision, context_recall, answer_similarity, answer_correctness, harmfulness], llm=llm_model, embeddings=embed_model)

In [ ]:
import numpy as np

result_dict = df.to_dict(orient='index')

# Convert any ndarray objects to list
for key in result_dict:
    for subkey in result_dict[key]:
        if isinstance(result_dict[key][subkey], (np.ndarray, list)):
            result_dict[key][subkey] = result_dict[key][subkey].tolist() if isinstance(result_dict[key][subkey], np.ndarray) else result_dict[key][subkey]

# Save the dictionary to a JSON file
with open("eval_results.json", "w") as file:
    json.dump(result_dict, file, indent=4)

In [ ]:
for key,val in score.items():
    print(f"{key}: {val}")